Module 11 Assignment
Lyn Nguyen 
11.14.2022

Code Resources: 
- Module 11 lecture (author piorkja1)
- https://neptune.ai/blog/pyldavis-topic-modelling-exploration-tool-that-every-nlp-data-scientist-should-know

In [3]:
# https://neptune.ai/blog/pyldavis-topic-modelling-exploration-tool-that-every-nlp-data-scientist-should-know

# First Part is to Read Reddit and store comments
import praw 
import gensim

# setup for tokenization and stopwords
from nltk.corpus import stopwords
from nltk.tokenize import wordpunct_tokenize, TweetTokenizer
from gensim import corpora, models, similarities
from nltk import ngrams
#from collections import defaultdict

stop_words = set(stopwords.words('english'))
stop_words.update(['.',  ',', '"', "'", '?', '!', ':', ';', '(', ')', '[', ']', '{', '}','year', # remove it if you need punctuation
'#', '://', '/', 'www', '-', 'com', '=', '...', 'org', 'https', '@', '&', "'", '"', 'msnbc', 'foxnews', 'npr', 'nytimes', 'cnn', 'usedgov']) # added for this assignment


In [4]:
# lyn - get input data and tokenize, try bike data from module 8 
import pandas as pd
import glob
# msg = df['Post Message']

df = pd.read_csv("data/master_annotated.csv")

In [5]:
df.columns

Index(['Unnamed: 0', 'experiment_id', 'experiment_group', 'text', 'tweet_id',
       'tweet_likes', 'retweets', 'tweet_created_at', 'user_id',
       'in_reply_to_status_id', 'in_reply_to_user_id',
       'in_reply_to_screen_name', 'dow', 'month_day', 'time', 'yr', 'ymd',
       'tweet_id_char', 'created_at', 'description', 'location',
       'followers_count', 'screen_name', 'statuses_count', 'favourites_count',
       'verified', 'user_id_char', 'text_length', 'text_word_count',
       'opinion_key', 'opinion_label', 'opinion_annotation_confidence',
       'ego_involvement_key', 'ego_involvement_label',
       'ego_involvement_annotation_confidence'],
      dtype='object')

In [6]:
# combine each post into a single "word" list of tokens
# 
import re
import advertools 
msg = df['text']
tk = TweetTokenizer()
all_tokens = []
list_of_tokens= []
two_gram = []

for post in msg:
    
    post = re.sub(r'[^\w\s]','',post) # madelaine

    # single 1gram token 
    words = [i.lower() for i in tk.tokenize(post) if i.lower() not in stop_words]

    # # 2ngram token 
    # # single token after removing stopwords back into a string 
    # text = " ".join(words)

    # # tokenize by 2-gram
    # for i in advertools.word_tokenize(text, 2):
    #     words.append(i)

    all_tokens.extend(words) # tokens in a single list
    list_of_tokens.append(words) # list of list of tokens

# words
texts = words

# # remove words that appear only once
tokens_once = set(words for words in set(all_tokens) if all_tokens.count(words) == 1)
token_often = [ w for w in all_tokens if w not in tokens_once ] 


print(len(tokens_once))
print(len(all_tokens))
print(len(token_often))



1220
7461
6241


In [8]:
token_often

# try to remove '"' and ''' 

    # text = re.sub(exotic_quotes, "'", text) # changing the second argument to fill the kind of quote you want to replace the exotic ones with
    # quote_replace_text.extend(text)


['maddowblog',
 'defense',
 'mean',
 'voters',
 'tax',
 'payers',
 'shouldnt',
 'pay',
 'someone',
 'elses',
 'student',
 'loan',
 'illegal',
 'maddowblog',
 'feel',
 'sorry',
 'suckers',
 'fell',
 'student',
 'loan',
 'forgiveness',
 'scam',
 'voted',
 'joe',
 'maddowblog',
 'setting',
 '2024',
 'elections',
 'republicans',
 'climate',
 'mass',
 'abortion',
 'student',
 'debt',
 'forgiveness',
 'maddowblog',
 'cant',
 'pay',
 'student',
 'loans',
 'getting',
 'job',
 'degree',
 'earned',
 'means',
 'time',
 'money',
 'much',
 'doesnt',
 'person',
 'poor',
 'choices',
 'shouldnt',
 'person',
 'bs',
 'debt',
 'pay',
 'maddowblog',
 'simple',
 'defense',
 'pay',
 'student',
 'loans',
 'everyone',
 'elses',
 'ridiculous',
 'biden',
 'pushing',
 'days',
 'maddowblog',
 'nothing',
 'simple',
 'student',
 'loan',
 'forgiveness',
 'unconstitutional',
 'money',
 'tax',
 'payers',
 'money',
 'billion',
 'afghanistan',
 'pay',
 'student',
 'loans',
 'end',
 'heres',
 'idea',
 'msm',
 'job',
 'st

In [9]:
# Prep tokens into corpora
common_dictionary = corpora.Dictionary(list_of_tokens)   # outputs tuple (index, number of occurrences)
common_corpus = [common_dictionary.doc2bow(text) for text in list_of_tokens]
id2word = common_dictionary


2022-12-09 17:22:13,310 | INFO | dictionary.py:199 | add_documents | adding document #0 to Dictionary(0 unique tokens: [])
2022-12-09 17:22:13,320 | INFO | dictionary.py:204 | add_documents | built Dictionary(2038 unique tokens: ['convincing', 'defense', 'elses', 'hardworking', 'illegal']...) from 468 documents (total 7461 corpus positions)
2022-12-09 17:22:13,320 | INFO | utils.py:447 | add_lifecycle_event | Dictionary lifecycle event {'msg': "built Dictionary(2038 unique tokens: ['convincing', 'defense', 'elses', 'hardworking', 'illegal']...) from 468 documents (total 7461 corpus positions)", 'datetime': '2022-12-09T17:22:13.320942', 'gensim': '4.1.2', 'python': '3.9.12 (main, Apr  5 2022, 01:53:17) \n[Clang 12.0.0 ]', 'platform': 'macOS-10.16-x86_64-i386-64bit', 'event': 'created'}


In [10]:
# tuple to (word, total occurrences)
[[(common_dictionary[i], freq) for i, freq in doc] for doc in common_corpus[:5]]

[[('convincing', 1),
  ('defense', 1),
  ('elses', 1),
  ('hardworking', 1),
  ('illegal', 1),
  ('loan', 1),
  ('maddowblog', 1),
  ('mean', 1),
  ('mention', 1),
  ('pay', 1),
  ('payers', 1),
  ('shouldnt', 1),
  ('simpletons', 1),
  ('someone', 1),
  ('student', 1),
  ('tax', 1),
  ('voters', 1)],
 [('loan', 1),
  ('maddowblog', 1),
  ('student', 1),
  ('feel', 1),
  ('fell', 1),
  ('forgiveness', 1),
  ('joe', 1),
  ('scam', 1),
  ('sorry', 1),
  ('suckers', 1),
  ('voted', 1)],
 [('maddowblog', 1),
  ('student', 1),
  ('forgiveness', 1),
  ('2024', 1),
  ('abortion', 1),
  ('climate', 1),
  ('debt', 1),
  ('elections', 1),
  ('mass', 1),
  ('republicans', 1),
  ('setting', 1),
  ('shootings', 1),
  ('tsunami', 1)],
 [('maddowblog', 1),
  ('pay', 2),
  ('shouldnt', 1),
  ('student', 1),
  ('debt', 1),
  ('bs', 1),
  ('cant', 1),
  ('choices', 1),
  ('degree', 1),
  ('doesnt', 1),
  ('earned', 1),
  ('getting', 1),
  ('job', 1),
  ('loans', 1),
  ('means', 1),
  ('money', 1),
  ('m

In [11]:
# https://neptune.ai/blog/pyldavis-topic-modelling-exploration-tool-that-every-nlp-data-scientist-should-know

from pprint import pprint
from gensim.models.ldamodel import LdaModel

lda_model = LdaModel(corpus=common_corpus,
                   id2word=id2word,
                   num_topics=5, 
                   random_state=0,
                   chunksize=100,
                   alpha='auto',
                   per_word_topics=True
                   )
pprint(lda_model.print_topics())
doc_lda = lda_model[common_corpus]

2022-12-09 17:22:21,471 | INFO | ldamodel.py:595 | init_dir_prior | using autotuned alpha, starting with [0.2, 0.2, 0.2, 0.2, 0.2]
2022-12-09 17:22:21,473 | INFO | ldamodel.py:576 | init_dir_prior | using symmetric eta at 0.2
2022-12-09 17:22:21,475 | INFO | ldamodel.py:481 | __init__ | using serial LDA version on this node
2022-12-09 17:22:21,481 | INFO | ldamodel.py:947 | update | running online (single-pass) LDA training, 5 topics, 1 passes over the supplied corpus of 468 documents, updating model once every 100 documents, evaluating perplexity every 468 documents, iterating 50x with a convergence threshold of 0.001000
2022-12-09 17:22:21,482 | WARNING | ldamodel.py:958 | update | too few updates, training might not converge; consider increasing the number of passes or iterations to improve accuracy
2022-12-09 17:22:21,482 | INFO | ldamodel.py:1001 | update | PROGRESS: pass 0, at document #100/468
2022-12-09 17:22:21,543 | INFO | ldamodel.py:794 | update_alpha | optimized alpha [0.2

[(0,
  '0.064*"student" + 0.033*"loan" + 0.030*"loans" + 0.022*"forgiveness" + '
  '0.018*"never" + 0.017*"like" + 0.014*"biden" + 0.013*"pay" + 0.012*"people" '
  '+ 0.011*"black"'),
 (1,
  '0.058*"student" + 0.040*"loan" + 0.038*"like" + 0.031*"votes" + '
  '0.029*"forgiveness" + 0.028*"vote" + 0.017*"buy" + 0.016*"buying" + '
  '0.015*"biden" + 0.014*"get"'),
 (2,
  '0.029*"student" + 0.026*"another" + 0.020*"loan" + 0.018*"forgiveness" + '
  '0.017*"promise" + 0.017*"bidens" + 0.017*"get" + 0.014*"debt" + '
  '0.013*"democrats" + 0.012*"scam"'),
 (3,
  '0.028*"student" + 0.020*"pay" + 0.018*"loans" + 0.014*"back" + 0.014*"pass" '
  '+ 0.013*"loan" + 0.011*"get" + 0.011*"court" + 0.010*"american" + '
  '0.010*"say"'),
 (4,
  '0.039*"debt" + 0.033*"student" + 0.026*"forgiveness" + 0.020*"loans" + '
  '0.019*"loan" + 0.010*"really" + 0.010*"college" + 0.009*"want" + '
  '0.008*"forgive" + 0.008*"pay"')]


In [50]:
# visualize 
!pip install pyLDAvis
import pyLDAvis.gensim_models # instead of from pyLDAvis import gensim_models
from pyLDAvis import display

topic_vis = pyLDAvis.gensim_models.prepare(lda_model, common_corpus, id2word) #dictionary)
display(topic_vis)


/Users/lnguyen/opt/anaconda3/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
/Users/lnguyen/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/lnguyen/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/lnguyen/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/lnguyen/opt/anaconda3/lib/